In [92]:
import numpy as np
import pandas as pd

def decide(probs=None, utils=None):
    if probs is None and utils is None:
        raise ValueError("Either 'probs' or 'utils' must be given")

    if utils is None:
        if len(probs.shape) > 1:
            raise ValueError("'probs' is not a vector, please specify array of utilities")
        utils = np.diag(probs)

    elif probs is None:
        probs = np.full(utils.shape[1], 1/utils.shape[1])

    if len(probs) != utils.shape[1]:
        raise ValueError('Mismatch between inference and utility variates')

    if not isinstance(utils, pd.DataFrame):
        utils = pd.DataFrame(utils)

    # Calculate expected utilities (matrix product)
    exputils = np.dot(utils, probs)
    # If expected utilites are the same for every class, choose random class
    if(len(set(exputils)) == 1):
        exputils = pd.Series(exputils, index=utils.index).sort_values(ascending=False)
        return {'EUs': exputils, 'optimal': random.randint(0,len(exputils)-1)}

        
    exputils = pd.Series(exputils, index=utils.index).sort_values(ascending=False)
    #Select one decision with max expected utility
    optimal = exputils.idxmax()
    return {'EUs': exputils, 'optimal': optimal}

# Example usage
probs = np.array([0.3, 0.7])
utils = np.array([[10, -10], [-10, 10]])
result = decide(probs=probs, utils=utils)
print(result)


{'EUs': 1    4.0
0   -4.0
dtype: float64, 'optimal': 1}


In [93]:
# Probabilities from RNopm infer
prob_predict_cat = [0.976039626021315, 0.0239603739786851]
prob_predict_dog = [0.00717048738691545, 0.992829512613085]

prob = np.array([prob_predict_cat, prob_predict_dog])
prob

array([[0.97603963, 0.02396037],
       [0.00717049, 0.99282951]])

In [94]:
#CM_df = pd.read_csv('C:\HVL\OneDrive - Høgskulen på Vestlandet\ADA511\Python\project\cd_dataframe.csv')
CM_df = pd.read_csv('/Users/eilertskram/Library/CloudStorage/OneDrive-HøgskulenpåVestlandet/ADA511/Python/project/cd_dataframe.csv')

In [95]:
CM_df.head()

,Unnamed: 0,True Cat,True Dog
0,Predicted Cat,1744,38
1,Predicted Dog,6,1712


In [96]:
CM = np.array([[1744, 38], [6, 1712]])

In [97]:
util = np.array([[1, -2], [-10, 1]]) 

In [98]:
expected_RN = np.multiply(CM, util)
expected_RN = expected_RN.sum()

In [99]:
#test_df = pd.read_csv('C:\\HVL\\OneDrive - Høgskulen på Vestlandet\\ADA511\\Python\\project\\test_dataframe.csv')
test_df = pd.read_csv('/Users/eilertskram/Library/CloudStorage/OneDrive-HøgskulenpåVestlandet/ADA511/Python/project/test_dataframe.csv')

In [100]:
cm_rn = np.array([[0, 0], [0, 0]]) 

In [101]:
def func_expected_opm(df,util, cm):
    expected_opm = 0
    number_of_overrides = 0

    for index, row in df.iterrows():
        predicted = row['Predicted']
        actual = row['Actual']

        if predicted == 'dog':
            optimal = decide(prob_predict_dog, util).get("optimal")
            
            if optimal == 1:
                if 'dog' == actual: #RPopm did not override and resnet predicted dog and it was dog
                    expected_opm += util[1][1]
                    cm[1][1] += 1
                else:   #RPopm did not override and resnet predicted dog and but it was cat
                    expected_opm += util[1][0]
                    cm[1][0] += 1
            else:
                number_of_overrides += 1
                if 'cat' == actual: #RPopm override and guessed cat and it was cat
                    expected_opm += util[0][0] 
                    cm[0][0] += 1  
                else: #RPopm override and guessed cat and but it was dog
                    expected_opm += util[0][1]
                    cm[0][1] += 1
        else:
            optimal = decide(prob_predict_cat, util).get("optimal")

            if optimal == 0:
                if 'cat' == actual: #RPopm did not override and resnet predicted cat and it was cat
                    expected_opm += util[0][0]
                    cm[0][0] += 1
                else:   #RPopm did not override and resnet predicted cat and but it was dog
                    expected_opm += util[0][1]
                    cm[0][1] += 1
            else:
                number_of_overrides += 1
                if 'dog' == actual: #RPopm override and guessed dog and it was dog
                    expected_opm += util[1][1]  
                    cm[1][1] += 1
                else: #RPopm override and guessed dog and but it was cat
                    expected_opm += util[1][0]
                    cm[1][0] += 1

    return expected_opm, number_of_overrides

    

            
        


## Trivial utility-matrix

In [102]:
print(f'Expected utilities with resnet alone: {np.multiply(CM, util).sum()}')
expected_opm = func_expected_opm(test_df, util, cm_rn)
print(f"Expected utilities with OPM: {expected_opm[0]}, total number over overrides {expected_opm[1]}")

Expected utilities with resnet alone: 3320


Expected utilities with OPM: 3320, total number over overrides 0


In [103]:
CM

array([[1744,   38],
       [   6, 1712]])

In [104]:
cm_rn

array([[1744,   38],
       [   6, 1712]])

## Slightly above threshold(p)

In [105]:
cm_rn = np.array([[0, 0], [0, 0]]) 

In [106]:
p = 0. + (1.-0.)*(0.98/(1-0.98))

In [107]:
u_above = np.array([[1.,0.], [0, p]]) #utility matrix using slightly above threshold(p)

In [108]:
print(f'Expected utilities with resnet alone: {np.multiply(CM, u_above).sum()}')
expected_opm = func_expected_opm(test_df, u_above, cm_rn)
print(f"Expected utilities with OPM: {expected_opm[0]}, total number over overrides {expected_opm[1]}")

Expected utilities with resnet alone: 85631.99999999993
Expected utilities with OPM: 85749.99999999999, total number over overrides 1782


In [109]:
CM

array([[1744,   38],
       [   6, 1712]])

In [110]:
cm_rn #confusion matrix 

array([[   0,    0],
       [1750, 1750]])

## Slightly below threshold(p)

In [111]:
cm_rn = np.array([[0, 0], [0, 0]]) 

In [112]:
p = 0. + (1.-0.)*(0.96/(1-0.96))

In [113]:
u_below = np.array([[1.,0.], [0, p]]) #utility matrix using sligthly below threshold(p)

In [114]:
print(f'Expected utilities with resnet alone: {np.multiply(CM, u_below).sum()}')
expected_opm = func_expected_opm(test_df, u_below, cm_rn)
print(f"Expected utilities with OPM: {expected_opm[0]}, total number over overrides {expected_opm[1]}")

Expected utilities with resnet alone: 42831.99999999996
Expected utilities with OPM: 42832.0, total number over overrides 0


In [115]:
CM

array([[1744,   38],
       [   6, 1712]])

In [116]:
cm_rn

array([[1744,   38],
       [   6, 1712]])

EQUAL


In [117]:
cm_rn = np.array([[0, 0], [0, 0]]) 

In [118]:
p = 0. + (1.-0.)*(0.98/(1-0.98))

In [119]:
u_eq = np.array([[0.,0.], [-p, p]]) #utility matrix using slightly above threshold(p)

In [120]:
print(f'Expected utilities with resnet alone: {np.multiply(CM, u_eq).sum()}')
expected_opm = func_expected_opm(test_df, u_eq, cm_rn)
print(f"Expected utilities with OPM: {expected_opm[0]}, total number over overrides {expected_opm[1]}")

Expected utilities with resnet alone: 83593.99999999993
Expected utilities with OPM: 83593.99999999999, total number over overrides 0


In [121]:
cm_rn

array([[1744,   38],
       [   6, 1712]])